# Classifying Audio Data using AST and different classification algorithm

Importing necessary packages and libraries

In [153]:
import functools
from pathlib import Path


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    f1_score,
    log_loss,
    recall_score,
)
from sklearn.model_selection import (
    cross_val_predict,
    StratifiedKFold,
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#self note: removed the skopt line below so it'll work, if it gives errors
from skopt import BayesSearchCV
# import torch
# import torchaudio
# from tqdm.notebook import tqdm


def to_prob(metric):
    @functools.wraps(metric)
    def metric_that_takes_prob(y_actual, y_pred, sample_weight=None):
        return metric(y_actual, y_pred.argmax(1), sample_weight=sample_weight)

    return metric_that_takes_prob


metrics = {
    "accuracy": to_prob(accuracy_score),
    "balanced_accuracy": to_prob(balanced_accuracy_score),
    "unweighted_f1": to_prob(functools.partial(f1_score, average="macro")),
    "UAR": to_prob(functools.partial(recall_score, average="macro")),
    "logloss": log_loss,
}

In [154]:
#create a dataframe to hold the data
df = pd.read_csv('new_train_test.csv')
#check labels
display(df.Label.value_counts())

Label
selftalk                  1885
frustrated                1536
delighted                 1272
dysregulated               704
social                     634
request                    419
affectionate               129
yes                        123
laughter                    93
dysregulation-sick          74
happy                       61
help                        24
more                        22
protest                     21
bathroom                    20
dysregulation-bathroom      18
no                          12
glee                         8
laugh                        8
tablet                       7
hunger                       4
greeting                     3
Name: count, dtype: int64

In [155]:
#Isolate the data for participant #05
df_P05 = df.loc[df.Participant == "P05"]
#check labels
display(df_P05.Label.value_counts())

Label
selftalk        286
frustrated      283
delighted       235
dysregulated    116
happy            61
help             24
laughter         13
request           6
hunger            4
Name: count, dtype: int64

In [156]:
label_counts = df_P05['Label'].value_counts()
labels_to_keep = label_counts[label_counts >= 30].index

# Filter the DataFrame to keep only the rows with these labels
df_P05_dropped = df_P05[df_P05['Label'].isin(labels_to_keep)]
df_P05_dropped['Label'].value_counts()

Label
selftalk        286
frustrated      283
delighted       235
dysregulated    116
happy            61
Name: count, dtype: int64

A dataframe is isolated to contain only the training data for participant 05

In [157]:
#create a dataframe that contains training data for participant 05
train_P05 = df_P05_dropped.loc[df_P05_dropped.is_test == 0]

In [158]:
#temporary cell: checking the dataframe
#print(train_P05)
print(train_P05.Label.value_counts())

Label
frustrated      208
selftalk        199
delighted       146
dysregulated     74
happy            46
Name: count, dtype: int64


In [159]:
train_P05.head()

,Unnamed: 0,Filename,Participant,Label,is_test,Session
2790,2790,200929_0101_00-02-06.53--00-02-07.10.wav,P05,happy,0,200929_0101
2791,2791,200929_2203_00-00-59.95--00-01-03.62.wav,P05,happy,0,200929_2203
2792,2792,200929_0101_00-06-53.65--00-06-54.43c.wav,P05,happy,0,200929_0101
2793,2793,200929_0101_00-06-52.47--00-06-52.91c.wav,P05,happy,0,200929_0101
2794,2794,200929_2203_00-06-05.52--00-06-06.36.wav,P05,happy,0,200929_2203


### Load the tensors derived from the AST model for inputting in different classification algorithms

In [160]:
X = np.loadtxt('X1_ast_numpy')

In [161]:
y = np.loadtxt('y_trainP5')

## Use Logistic Regression for the Classification

In [162]:
est = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        max_iter=10**6,
    ),
)
opt = BayesSearchCV(
    est,
    {
        "logisticregression__C": (5e-3, 1, "log-uniform"),
    },
    n_iter=20,
    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=12345),
    scoring="accuracy",
)
opt.fit(
    X.reshape(len(X), -1),
    y,
)
print(opt.best_params_)
print("Best accuracy:", opt.best_score_)

OrderedDict({'logisticregression__C': 0.15905067752712518})
Best accuracy: 0.6153204565408253


In [163]:
train_P05["session"] = train_P05.Filename.apply(
    lambda name: name.split("-")[0][:-3]
)

C:\Users\squis\AppData\Local\Temp\ipykernel_32800\3423150322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_P05["session"] = train_P05.Filename.apply(


In [164]:
session_weight = (
    (1 / train_P05.session.value_counts())
    .clip(None, 0.1)
    .loc[train_P05.session]
).values
session_and_label_weight = (
    1
    / pd.Series(session_weight, train_P05.index)
    .groupby(train_P05.Label)
    .sum()
).loc[train_P05.Label].values * session_weight

In [165]:
est = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        C=opt.best_params_["logisticregression__C"],
        max_iter=10**6,
    ),
)
oos_pred_prob = cross_val_predict(
    est,
    X.reshape(len(X), -1),
    y,
    cv=StratifiedKFold(
        n_splits=10,
        shuffle=True,
        random_state=1234,  # Using different seed to avoid over-fitting parameter
    ),
    method="predict_proba",
    #params={"logisticregression__sample_weight": session_weight},
)
oos_pred = oos_pred_prob.argmax(1)

display(
    pd.Series(
        {name: metric(y, oos_pred_prob) for name, metric in metrics.items()},
        name="no_weight",
    ).round(3)
)
display(
    pd.Series(
        {
            name: metric(y, oos_pred_prob, sample_weight=session_weight)
            for name, metric in metrics.items()
        },
        name="session_weight",
    ).round(3)
)
display(
    pd.Series(
        {
            name: metric(
                y, oos_pred_prob, sample_weight=session_and_label_weight
            )
            for name, metric in metrics.items()
        },
        name="session_and_label_weight",
    ).round(3)
)

accuracy             0.608
balanced_accuracy    0.547
unweighted_f1        0.547
UAR                  0.547
logloss              1.007
Name: no_weight, dtype: float64

accuracy             0.593
balanced_accuracy    0.544
unweighted_f1        0.545
UAR                  0.544
logloss              1.057
Name: session_weight, dtype: float64

accuracy             0.544
balanced_accuracy    0.544
unweighted_f1        0.529
UAR                  0.544
logloss              1.196
Name: session_and_label_weight, dtype: float64

## Use RandomForestClassifier

In [166]:
from skopt.space import Categorical
from sklearn.ensemble import RandomForestClassifier

In [167]:
est = make_pipeline(
    StandardScaler(),
    RandomForestClassifier()
)

# Define the hyperparameter grid for RandomForestClassifier
opt = BayesSearchCV(
    est,
    {
        "randomforestclassifier__n_estimators": [800, 1000, 1200],  # Number of trees in the forest
        "randomforestclassifier__max_depth": [3,5,7],  # Maximum depth of the trees
    },
    n_iter=20,
    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=12345),
    scoring="accuracy",
)

opt.fit(
    X.reshape(len(X), -1),
    y,
)
print(opt.best_params_)
print("Best accuracy:", opt.best_score_)

c:\Users\squis\miniforge3\envs\deep-learning\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [7, 800] before, using random point [5, 800]
  warnings.warn(
c:\Users\squis\miniforge3\envs\deep-learning\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [7, 800] before, using random point [3, 1000]
  warnings.warn(
c:\Users\squis\miniforge3\envs\deep-learning\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [7, 800] before, using random point [7, 800]
  warnings.warn(
c:\Users\squis\miniforge3\envs\deep-learning\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [7, 800] before, using random point [3, 800]
  warnings.warn(
c:\Users\squis\miniforge3\envs\deep-learning\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated 

OrderedDict({'randomforestclassifier__max_depth': 7, 'randomforestclassifier__n_estimators': 1200})
Best accuracy: 0.6242098331870062


In [168]:
est = make_pipeline(
    StandardScaler(),
    RandomForestClassifier(
        n_estimators=opt.best_params_.get("randomforestclassifier__n_estimators", 100),
        max_depth=opt.best_params_.get("randomforestclassifier__max_depth", None),
        random_state=12345  # You can include a random state for reproducibility
    ),
)

# Cross-validation prediction probabilities
oos_pred_prob = cross_val_predict(
    est,
    X.reshape(len(X), -1),
    y,
    cv=StratifiedKFold(
        n_splits=10,
        shuffle=True,
        random_state=1234,  # Using a different seed to avoid overfitting parameter
    ),
    method="predict_proba",
    #fit_params={"randomforestclassifier__sample_weight": session_weight},
)

# Get the predicted classes from the probabilities
oos_pred = oos_pred_prob.argmax(1)

# Display performance metrics
display(
    pd.Series(
        {name: metric(y, oos_pred_prob) for name, metric in metrics.items()},
        name="no_weight",
    ).round(3)
)
display(
    pd.Series(
        {
            name: metric(y, oos_pred_prob, sample_weight=session_weight)
            for name, metric in metrics.items()
        },
        name="session_weight",
    ).round(3)
)
display(
    pd.Series(
        {
            name: metric(
                y, oos_pred_prob, sample_weight=session_and_label_weight
            )
            for name, metric in metrics.items()
        },
        name="session_and_label_weight",
    ).round(3)
)

accuracy             0.620
balanced_accuracy    0.518
unweighted_f1        0.507
UAR                  0.518
logloss              1.016
Name: no_weight, dtype: float64

accuracy             0.608
balanced_accuracy    0.520
unweighted_f1        0.518
UAR                  0.520
logloss              1.037
Name: session_weight, dtype: float64

accuracy             0.520
balanced_accuracy    0.520
unweighted_f1        0.478
UAR                  0.520
logloss              1.188
Name: session_and_label_weight, dtype: float64

## Use XGBoostClassifier

In [169]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [170]:
# Create the pipeline with XGBClassifier
estxgbast = make_pipeline(
    StandardScaler(),
    XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')  # XGBoost-specific parameters
)

# Define the hyperparameter grid for XGBClassifier
param_gridxgbast = {
    "xgbclassifier__n_estimators": [800, 1000, 1200],  # Number of boosting rounds
    "xgbclassifier__max_depth": [3, 5, 7],  # Maximum depth of the tree
    "xgbclassifier__learning_rate": [0.01, 0.1, 0.2],  # Step size shrinkage
}

# Set up the RandomizedSearchCV
optxgb = GridSearchCV(
    estxgbast,
    param_grid=param_gridxgbast,
    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=12345),
    scoring="accuracy",
    n_jobs=-1
)

# Now use `opt` for hyperparameter tuning
optxgb.fit(
    X.reshape(len(X), -1),
    y,
)

c:\Users\squis\miniforge3\envs\deep-learning\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:58:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=12345, shuffle=True),
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('xgbclassifier',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      device=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_...
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'xgbclassifier__learning_rate': [0.01, 0.1, 0.2],
                         'xgbclassifier__max_depth': [3, 5, 7],
                         'xgbclassifier__n_estimators': [800, 1000, 1200]},
             scoring='accuracy')

In [171]:
print(optxgb.best_params_)
print("Best accuracy:", optxgb.best_score_)

{'xgbclassifier__learning_rate': 0.1, 'xgbclassifier__max_depth': 5, 'xgbclassifier__n_estimators': 1000}
Best accuracy: 0.6344600526777875


In [174]:
est = make_pipeline(
    StandardScaler(),
    XGBClassifier(
        n_estimators=optxgb.best_params_["xgbclassifier__n_estimators"],  # Example of using best params
        max_depth=optxgb.best_params_["xgbclassifier__max_depth"],
        learning_rate=optxgb.best_params_["xgbclassifier__learning_rate"]
    ),
)

# Cross-validation predictions with probability estimates
oos_pred_prob = cross_val_predict(
    est,
    X.reshape(len(X), -1),
    y,
    cv=StratifiedKFold(
        n_splits=10,
        shuffle=True,
        random_state=1234,  # Using different seed to avoid over-fitting parameter
    ),
    method="predict_proba",
    #fit_params={"xgbclassifier__sample_weight": session_weight},
)

# Get the predicted class by taking the argmax of the predicted probabilities
oos_pred = oos_pred_prob.argmax(1)

# print(f"{oos_pred_prob.shape=}, {oos_pred.shape=}")

# print(f"Accuracy: {accuracy_score(y, oos_pred):.3f}")
# print(f"Unweighted F1 score: {f1_score(y, oos_pred, average='macro'):.3f}")
# print(f"Cross-entropy: {log_loss(y, oos_pred_prob):.3f}")

# # Display metrics without sample weight
display(
    pd.Series(
        {name: metric(y, oos_pred_prob) for name, metric in metrics.items()},
        name="no_weight",
    ).round(3)
)

# Display metrics with session weight
display(
    pd.Series(
        {
            name: metric(y, oos_pred_prob, sample_weight=session_weight)
            for name, metric in metrics.items()
        },
        name="session_weight",
    ).round(3)
)

# Display metrics with session and label weight
display(
    pd.Series(
        {
            name: metric(
                y, oos_pred_prob, sample_weight=session_and_label_weight
            )
            for name, metric in metrics.items()
        },
        name="session_and_label_weight",
    ).round(3)
)

accuracy             0.630
balanced_accuracy    0.569
unweighted_f1        0.575
UAR                  0.569
logloss              1.274
Name: no_weight, dtype: float64

accuracy             0.608
balanced_accuracy    0.561
unweighted_f1        0.561
UAR                  0.561
logloss              1.264
Name: session_weight, dtype: float64

accuracy             0.561
balanced_accuracy    0.561
unweighted_f1        0.547
UAR                  0.561
logloss              1.455
Name: session_and_label_weight, dtype: float64

## Testing set

In [179]:
# Instantiate the XGBClassifier with the best parameters
bestxgb = XGBClassifier(
    learning_rate=0.1, 
    max_depth=5, 
    n_estimators=1000,
    use_label_encoder=False,  # XGBoost-specific parameter
    eval_metric='mlogloss'    # XGBoost-specific parameter
)

# Create the pipeline with StandardScaler and the instantiated bestxgb
model = make_pipeline(
    StandardScaler(),
    bestxgb  # Pass the already instantiated XGBClassifier object
)


# # # Now use `opt` for hyperparameter tuning
model.fit(
    X.reshape(len(X), -1),
    y,
)

c:\Users\squis\miniforge3\envs\deep-learning\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:40:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='mlogloss',
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=1000, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [180]:
X1_test = X = np.loadtxt('X1test_numpy')

y_test = np.loadtxt('ytest_numpy')

In [181]:
# Predict the probabilities for the test set
probabilities = model.predict_proba(X1_test)

# Predict the class labels for the test set
predictions = model.predict(X1_test) ## Should X_test should be replaced with X_test.reshape(len(X_test), -1) ?



#metrics

#checking accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')

# Compute the balanced accuracy score
balanced_accuracy = balanced_accuracy_score(y_test, predictions)
print(f'Balanced Accuracy Score: {balanced_accuracy}')

f1 = f1_score(y_test, predictions, average='macro')  
print(f'F1 Score (Macro): {f1}')

cm = confusion_matrix(y_test, predictions)
print(f'Confusion Matrix:\n{cm}')

Accuracy: 0.538961038961039
Balanced Accuracy Score: 0.5241352004575561
F1 Score (Macro): 0.49707752060875077
Confusion Matrix:
[[ 9  0  1  2  3]
 [ 0 62  4  6  3]
 [ 4 11  9  7 11]
 [ 5  8  2 52 20]
 [ 3 34  6 12 34]]
